In [12]:
import datetime as dt 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
team_df = pd.read_csv('bball_ref_map.csv')
team_df.index = team_df.bball_ref_alias

In [14]:
team_df

,Franchise,Lg,From,To,Plyfs,Div,Conf,Champ,bball_ref_alias,team_id
bball_ref_alias,,,,,,,,,,
ATL,Atlanta Hawks,NBA,1969,2019,33,5,0,0,ATL,sr:competitor:3423
STL,St. Louis Hawks,NBA,1956,1968,12,6,0,1,STL,sr:competitor:3423
MLH,Milwaukee Hawks,NBA,1952,1955,0,0,0,0,MLH,sr:competitor:3423
TRI,Tri-Cities Blackhawks,NBA,1950,1951,1,0,0,0,TRI,sr:competitor:3423
BOS,Boston Celtics,NBA/BAA,1947,2019,55,31,9,17,BOS,sr:competitor:3422
BRK,Brooklyn Nets,NBA,2013,2019,3,0,0,0,BRK,sr:competitor:3436
NJN,New Jersey Nets,NBA,1978,2012,16,4,2,0,NJN,sr:competitor:3436
NYN,New York Nets,NBA,1977,1977,0,0,0,0,NYN,sr:competitor:3436
CHO,Charlotte Hornets,NBA,2015,2019,1,0,0,0,CHO,sr:competitor:3430


In [ ]:
begin_years = team_df['From']
end_years = team_df['To']
team_ids = team_df['team_id']
team_alias_list = list(team_df.index)

In [ ]:
df = pd.DataFrame()

#loop through teams
for team in team_alias_list:
    
    # For each team, set the first and last year
    start_year = begin_years[team]
    end_year = end_years[team] 
    current_year = start_year     
       
    # Loop through the appropriate years for each team
    while current_year <= end_year:
        
        # Print current team/year create string
        print(team, ' ',str(current_year))
        
        # Change url string to team and year 
        url_string= 'https://www.basketball-reference.com/teams/' + team + '/' + str(current_year) + '_games.html'    

        # Read in the page and make the soup
        page = urlopen(url_string).read()
        soup = BeautifulSoup(page)

        ##############################################
        ### Find relevant tables, id features you want
        table = soup.find("div", {"id": "content"}) 
        pre_df = dict()
        features_wanted =  {'team_id','date_game','opp_name','game_location','game_result','pts','opp_pts'}
        ##############################################
        
        # Continue if a table is found
        if table != None:
            
            # Retreive all rows in table
            rows = table.find_all('tr')
            # Iterate over rows
            for row in rows:
                
                # For each row that exists iterate over features
                if (row.find('th', {"scope":"row"}) != None):                    
                    for f in features_wanted:             
                        
                        # team_id is not from b-ref
                        if f != 'team_id':
                            
                            # Find cell that contains the stat you want
                            cell = row.find("td",{"data-stat": f})
                            
                            # Encoding
                            try:                
                                a = cell.text.strip().encode()
                            except:
                                a = '-' 
                            try:
                                text = a.decode("utf-8")
                            except:
                                text = a 
                            
                            # Format game location
                            if f == 'game_location':
                                if text == '':
                                    text = 'home'
                                elif text == '@':
                                    text = 'away'
                                elif text == 'N':
                                    text = 'neutral'
                                else:
                                    text = ''
                                    
                        # if the feature is the team_id, get it from the list
                        else:
                            text = team_ids[team]
                        
                        # Create dataframe with scraped data 
                        if f in pre_df:
                             pre_df[f].append(text)
                        else:
                             pre_df[f]=[text]

                    temp_df = pd.DataFrame.from_dict(pre_df)
        
        # Add data to the main dataframe
        df = df.append(temp_df)
        # Move to the next year 
        current_year = current_year + 1 

In [ ]:
df.shape 

In [ ]:
df.head()